# CAT DATA VISUALISATION

By Min Yan. As at Tuesday, 29 Mar 2016.


# 1. Import Data 

## 1.1 Visualisation of Raw Data (acctt) - Accountants

In [1]:
import pandas as pd

acctt = pd.read_csv('ACC.csv')
accts = acctt['ACC'].unique() 
print accts, len(accts) # print unique accountants and total no

acctt

['ANGELA' 'CAIYEAN' 'CHWEELIAN' 'DINA' 'GERALDINE' 'HOAYBEE' 'JASON'
 'JOSEPH' 'RAYMOND' 'SEETLEE' 'SERENE' 'SIEWJIN'] 12


,Accountant,ACC,RANK,RATE,PRODUCTIVITY
0,Angela,ANGELA,Junior,15,31.60
1,Cai Yean,CAIYEAN,Senior,35,71.60
2,Chwee Lian,CHWEELIAN,Middle,25,50.00
3,Dina,DINA,Senior,35,67.79
4,Geraldine,GERALDINE,Senior,35,66.18
5,Hoay Bee,HOAYBEE,Middle,25,38.62
6,Jason,JASON,Junior,15,42.85
7,Joseph,JOSEPH,Junior,15,40.91
8,Raymond,RAYMOND,Junior,15,39.98
9,Seet Lee,SEETLEE,Senior,35,86.82


## 1.2 Visualisation of Raw Data (projt) - Project Breakdown

In [2]:
projt = pd.read_csv('PI.csv')
projt.drop(['HR'], axis=1, inplace=True)
projt.rename(columns={'HR_ROUND': 'HR'}, inplace=True)

projt.head(10)

,PROJ,FEE,Accountant,PROD,HR
0,3H aircon,3900,Angela,31.60,29.5
1,3H aircon,3900,Cai Yean,71.60,35.5
2,3H aircon,3900,Seet Lee,86.82,6.0
3,Active Links Airconditioning Engrg,2900,Angela,31.60,25.5
4,Active Links Airconditioning Engrg,2900,Seet Lee,86.82,6.0
5,Active Links Airconditioning Engrg,2900,Siew Jin,47.18,3.5
6,ADCT Technologies,6500,Raymond,39.98,42.0
7,ADCT Technologies,6500,Seet Lee,86.82,11.5
8,ADCT Technologies,6500,Joseph,40.91,16.0
9,ADCT Technologies,6500,Siew Jin,47.18,19.5


## 1.3 Capitalise projt 's accountant names

In [3]:
projt = pd.merge(projt, acctt[['Accountant','ACC']], how='left', on=['Accountant'])
# print projt.head()

projt.drop(['Accountant'], axis=1, inplace=True)
projt.head()

,PROJ,FEE,PROD,HR,ACC
0,3H aircon,3900,31.60,29.5,ANGELA
1,3H aircon,3900,71.60,35.5,CAIYEAN
2,3H aircon,3900,86.82,6.0,SEETLEE
3,Active Links Airconditioning Engrg,2900,31.60,25.5,ANGELA
4,Active Links Airconditioning Engrg,2900,86.82,6.0,SEETLEE


# 2. Generate Summary Stats for Data Cleaning

## 2.1 Count unique projects (& No. of accountants for each proj)

In [4]:
projects = projt['PROJ'].unique()
print "No. of unique projs:", len(projects)
print projt['PROJ'].value_counts().head(10)
projt['PROJ'].value_counts().tail(10)

No. of unique projs: 97
Linear Precision                   5
Jia Yi Air-Conditioning            5
Linear Tooling                     5
Ultrachem (S) Private Limited      5
Central Granite Industries         4
Perma Shipping Line (Singapore)    4
Netball Singapore                  4
Cicada Private Limited             4
ADCT Technologies                  4
Aldon Technologies Services        4
Name: PROJ, dtype: int64


Changi Sailing Club                  1
Raffles Treats Pte Limited           1
Pace Flow Control                    1
SSA_Water Polo                       1
World Auto                           1
Chelliah & Kiang                     1
Innochem                             1
Bachmann                             1
StarChase Motorsports (Singapore)    1
Institute of Shipping Management     1
Name: PROJ, dtype: int64

## 2.2 Denormalize projt table by compiling accountant names (delimiter = ', ')

In [5]:
# http://stackoverflow.com/questions/20625582/how-to-deal-with-this-pandas-warning
pd.options.mode.chained_assignment = None  # default='warn'

print "prev row count:", projt.shape
# http://stackoverflow.com/questions/27298178/concatenate-strings-from-several-rows-using-pandas-groupby
projt['ACCS'] = projt[['PROJ','ACC']].groupby(['PROJ'])['ACC'].transform(lambda x: ', '.join(x))
projt['T_HR'] = projt[['PROJ','HR']].groupby(['PROJ'])['HR'].transform(lambda x: sum(x))

result = projt.drop_duplicates(['PROJ'])
result.drop(['ACC','HR'], axis=1, inplace=True)
print "post row count:", result.shape
result.head(10)

prev row count: (242, 5)
post row count: (97, 5)


,PROJ,FEE,PROD,ACCS,T_HR
0,3H aircon,3900,31.60,"ANGELA, CAIYEAN, SEETLEE",71.0
3,Active Links Airconditioning Engrg,2900,31.60,"ANGELA, SEETLEE, SIEWJIN",35.0
6,ADCT Technologies,6500,39.98,"RAYMOND, SEETLEE, JOSEPH, SIEWJIN",89.0
10,Additions Lifestyle,3900,71.60,"CAIYEAN, ANGELA, RAYMOND",83.5
13,Aidha Ltd,1900,38.62,"HOAYBEE, SEETLEE, JASON",75.0
16,Aim Aircon Engineering,3000,39.98,"RAYMOND, CAIYEAN, ANGELA",88.5
19,Aim Fire Systems Engineering,3000,71.60,"CAIYEAN, ANGELA, SEETLEE",57.5
22,Al- us Enterprises,500,86.82,"SEETLEE, JASON, JOSEPH",16.0
25,Alan Telecom,1200,71.60,"CAIYEAN, HOAYBEE",12.5
27,Aldon Technologies Services,7500,39.98,"RAYMOND, JOSEPH, SEETLEE, SIEWJIN",302.0


## 2.3 Check for employee duplicates within projt

In [6]:
# print projects with duplicate employees
duplicate = False

for x in range(len(result)):
    acclist = result['ACCS'].iloc[x].split(", ")
    if len(acclist) != len(set(acclist)):
        duplicate = True
        print result.PROJ.iloc[x], acclist
        
if not duplicate:
    print "There are no employee duplicates for each project."

There are no employee duplicates for each project.


## 2.4 Calc Project Prod Index, Total Cost, Total Accountants

In [7]:
def get_hr(proj, acc):
    return projt[projt.PROJ == proj][projt.ACC == acc].iloc[0]['HR']
def get_rate(acc):
    return acctt[acctt.ACC == acc].iloc[0]['RATE']
def get_total_cost(proj):
    total = 0
    for acc in accts: 
        total += result[result.PROJ == proj].iloc[0][acc+'_COST'] # assume indiv costs are calculated
    return total

In [8]:
for acc in accts:
    result[acc] = [1 if acc in x else 0 for x in result.ACCS]
    string = acc + '_HR'
    result[acc+'_HR'] = [get_hr(result.PROJ.iloc[x],acc) if result[acc].iloc[x] == 1 else 0 for x in range(len(result))]
    rate = get_rate(acc)
    result[acc+'_COST'] = [hr * rate for hr in result[acc+'_HR']]
    
result['T_PROD'] = [result['FEE'].iloc[x] / result['T_HR'].iloc[x] for x in range(len(result))]
result['T_ACC'] = [len(accs.split(", ")) for accs in result.ACCS]
result['T_COST'] = [get_total_cost(proj) for proj in result.PROJ]
result['T_PROFIT'] = [result['FEE'].iloc[x] - result['T_COST'].iloc[x] for x in range(len(result))]
result['PERC_PROFIT'] = [result['T_PROFIT'].iloc[x] / result['FEE'].iloc[x] * 100 for x in range(len(result))]
    
accts_extended = []
for acc in accts:
    accts_extended.append(acc)
    accts_extended.append(acc+'_HR')
    accts_extended.append(acc+'_COST')
print accts_extended
result.head()

['ANGELA', 'ANGELA_HR', 'ANGELA_COST', 'CAIYEAN', 'CAIYEAN_HR', 'CAIYEAN_COST', 'CHWEELIAN', 'CHWEELIAN_HR', 'CHWEELIAN_COST', 'DINA', 'DINA_HR', 'DINA_COST', 'GERALDINE', 'GERALDINE_HR', 'GERALDINE_COST', 'HOAYBEE', 'HOAYBEE_HR', 'HOAYBEE_COST', 'JASON', 'JASON_HR', 'JASON_COST', 'JOSEPH', 'JOSEPH_HR', 'JOSEPH_COST', 'RAYMOND', 'RAYMOND_HR', 'RAYMOND_COST', 'SEETLEE', 'SEETLEE_HR', 'SEETLEE_COST', 'SERENE', 'SERENE_HR', 'SERENE_COST', 'SIEWJIN', 'SIEWJIN_HR', 'SIEWJIN_COST']


/Users/behminyan/anaconda2/lib/python2.7/site-packages/pandas/core/frame.py:1997: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  "DataFrame index.", UserWarning)


,PROJ,FEE,PROD,ACCS,T_HR,ANGELA,ANGELA_HR,ANGELA_COST,CAIYEAN,CAIYEAN_HR,...,SERENE_HR,SERENE_COST,SIEWJIN,SIEWJIN_HR,SIEWJIN_COST,T_PROD,T_ACC,T_COST,T_PROFIT,PERC_PROFIT
0,3H aircon,3900,31.60,"ANGELA, CAIYEAN, SEETLEE",71.0,1,29.5,442.5,1,35.5,...,0,0,0,0.0,0.0,54.929577,3,1895.0,2005.0,51.410256
3,Active Links Airconditioning Engrg,2900,31.60,"ANGELA, SEETLEE, SIEWJIN",35.0,1,25.5,382.5,0,0.0,...,0,0,1,3.5,87.5,82.857143,3,680.0,2220.0,76.551724
6,ADCT Technologies,6500,39.98,"RAYMOND, SEETLEE, JOSEPH, SIEWJIN",89.0,0,0.0,0.0,0,0.0,...,0,0,1,19.5,487.5,73.033708,4,1760.0,4740.0,72.923077
10,Additions Lifestyle,3900,71.60,"CAIYEAN, ANGELA, RAYMOND",83.5,1,31.0,465.0,1,27.0,...,0,0,0,0.0,0.0,46.706587,3,1792.5,2107.5,54.038462
13,Aidha Ltd,1900,38.62,"HOAYBEE, SEETLEE, JASON",75.0,0,0.0,0.0,0,0.0,...,0,0,0,0.0,0.0,25.333333,3,1690.0,210.0,11.052632


In [9]:
result.drop(accts_extended, axis=1).head(15) ## with less columns

,PROJ,FEE,PROD,ACCS,T_HR,T_PROD,T_ACC,T_COST,T_PROFIT,PERC_PROFIT
0,3H aircon,3900,31.60,"ANGELA, CAIYEAN, SEETLEE",71.0,54.929577,3,1895.0,2005.0,51.410256
3,Active Links Airconditioning Engrg,2900,31.60,"ANGELA, SEETLEE, SIEWJIN",35.0,82.857143,3,680.0,2220.0,76.551724
6,ADCT Technologies,6500,39.98,"RAYMOND, SEETLEE, JOSEPH, SIEWJIN",89.0,73.033708,4,1760.0,4740.0,72.923077
10,Additions Lifestyle,3900,71.60,"CAIYEAN, ANGELA, RAYMOND",83.5,46.706587,3,1792.5,2107.5,54.038462
13,Aidha Ltd,1900,38.62,"HOAYBEE, SEETLEE, JASON",75.0,25.333333,3,1690.0,210.0,11.052632
16,Aim Aircon Engineering,3000,39.98,"RAYMOND, CAIYEAN, ANGELA",88.5,33.898305,3,2337.5,662.5,22.083333
19,Aim Fire Systems Engineering,3000,71.60,"CAIYEAN, ANGELA, SEETLEE",57.5,52.173913,3,1552.5,1447.5,48.250000
22,Al- us Enterprises,500,86.82,"SEETLEE, JASON, JOSEPH",16.0,31.250000,3,260.0,240.0,48.000000
25,Alan Telecom,1200,71.60,"CAIYEAN, HOAYBEE",12.5,96.000000,2,337.5,862.5,71.875000
27,Aldon Technologies Services,7500,39.98,"RAYMOND, JOSEPH, SEETLEE, SIEWJIN",302.0,24.834437,4,7470.0,30.0,0.400000


# 3. Preliminary Analysis

## 3.1 Projs with only 1 accountant 

In [10]:
result[result.T_ACC == 1].drop(accts_extended, axis=1).drop(['T_COST','T_HR'],axis=1).sort_values(by='ACCS')

,PROJ,FEE,PROD,ACCS,T_PROD,T_ACC,T_PROFIT,PERC_PROFIT
142,Pace Flow Control,2900,71.60,CAIYEAN,71.604938,1,1482.5,51.120690
158,Raffles Treats Pte Limited,2100,71.60,CAIYEAN,71.186441,1,1067.5,50.833333
58,Changi Sailing Club,4500,50.00,CHWEELIAN,50.000000,1,2250.0,50.000000
90,Fong Foundation,4500,67.79,DINA,67.669173,1,2172.5,48.277778
206,SSF,9200,67.79,DINA,67.896679,1,4457.5,48.451087
205,SSA_Water Polo,1000,67.79,DINA,66.666667,1,475.0,47.500000
57,CGI Ind,4300,66.18,GERALDINE,66.153846,1,2025.0,47.093023
123,Lingfine Jewellery,2200,66.18,GERALDINE,66.666667,1,1045.0,47.500000
207,StarChase Motorsports (Singapore),1300,38.62,HOAYBEE,38.805970,1,462.5,35.576923
143,Paveway Explorer Holidays,1600,38.62,HOAYBEE,38.554217,1,562.5,35.156250


## 3.2 Projs making a loss

In [11]:
result[result.PERC_PROFIT < 0].drop(accts_extended, axis=1).drop(['T_COST','PROD'],axis=1)

,PROJ,FEE,ACCS,T_HR,T_PROD,T_ACC,T_PROFIT,PERC_PROFIT
34,Ariva Hospitality,2500,"DINA, SEETLEE, SERENE",96.5,25.906736,3,-877.5,-35.100000
72,Domino's Pizza Singapore,7900,"DINA, CAIYEAN",259.0,30.501931,2,-1165.0,-14.746835
98,Hock Chuan Kiat,4500,"DINA, HOAYBEE",170.5,26.392962,2,-837.5,-18.611111


## 3.3 Overall Summary Stats

note T_PROD: total productivity column

In [12]:
result.drop(accts_extended, axis=1).describe()

,FEE,PROD,T_HR,T_PROD,T_ACC,T_COST,T_PROFIT,PERC_PROFIT
count,97.000000,97.000000,97.00000,97.000000,97.000000,97.000000,97.000000,97.000000
mean,3415.979381,56.604330,74.71134,51.871033,2.494845,2037.061856,1378.917526,44.204857
std,2302.574944,18.568138,61.30507,18.672645,1.011638,1827.436053,1221.947883,21.825438
min,500.000000,31.600000,11.50000,24.834437,1.000000,210.000000,-1165.000000,-35.100000
25%,1500.000000,38.620000,28.00000,38.554217,2.000000,720.000000,570.000000,31.569767
50%,2900.000000,66.180000,62.00000,48.888889,3.000000,1552.500000,1067.500000,48.038462
75%,4500.000000,71.600000,96.50000,64.285714,3.000000,2735.000000,2005.000000,59.657895
max,10400.000000,86.820000,302.00000,107.407407,5.000000,9065.000000,5847.500000,81.896552


## 3.4 Retrieve top n projects (highest T_PROD, total productivity)

n=10 in this case

In [13]:
prods = sorted(result.T_PROD.unique(), reverse=True)

def get_top_n_projects(n): # in terms of productivity
    return result[result.T_PROD >= prods[n]].drop(accts_extended, axis=1).drop(['PROD'],axis=1).sort_values(by='T_PROD',ascending=False)
get_top_n_projects(10)

,PROJ,FEE,ACCS,T_HR,T_PROD,T_ACC,T_COST,T_PROFIT,PERC_PROFIT
96,Handball Federation,2900,"RAYMOND, CAIYEAN",27.0,107.407407,2,525.0,2375.0,81.896552
25,Alan Telecom,1200,"CAIYEAN, HOAYBEE",12.5,96.000000,2,337.5,862.5,71.875000
196,Skyline Navigation,3900,"CAIYEAN, DINA, SEETLEE",41.5,93.975904,3,1452.5,2447.5,62.756410
218,Taiyo Asset Management,2500,"ANGELA, CAIYEAN",27.5,90.909091,2,822.5,1677.5,67.100000
159,Red Sea and Gulf,1200,"SEETLEE, ANGELA",13.5,88.888889,2,252.5,947.5,78.958333
236,Underwater Technology Services (S),9500,SEETLEE,109.5,86.757991,1,3832.5,5667.5,59.657895
101,Institute of Estate Agents,2900,"HOAYBEE, SEETLEE",33.5,86.567164,2,857.5,2042.5,70.431034
182,Singapore Silat Federation,5000,"JASON, JOSEPH, SIEWJIN",58.0,86.206897,3,1000.0,4000.0,80.000000
3,Active Links Airconditioning Engrg,2900,"ANGELA, SEETLEE, SIEWJIN",35.0,82.857143,3,680.0,2220.0,76.551724
147,Perdana Parkcity,1500,"CAIYEAN, SEETLEE",18.5,81.081081,2,647.5,852.5,56.833333


In [14]:
# # Deprecated method

# def get_row(prod):
#     return result.drop(accts_extended, axis=1).drop(['PROD','ACCS','T_ACC'], axis=1)[result.T_PROD == prod]

# df = pd.DataFrame(columns=('PROJ','FEE','T_HR','T_PROD','T_COST','T_PROFIT','PERC_PROFIT'))
# for i in range(10):
#     row = get_row(prods[i])
#     print row
#     # df.append(row, ignore_index=True)
# print df

## 3.5 List of Projs under each accountant

In [15]:
acctsummary = pd.DataFrame() #create new DataFrame
acctsummary['ACC'] = accts

temp=pd.DataFrame()
temp['PROJS'] = projt[['ACC','PROJ']].sort_values(by=['ACC','PROJ']).groupby(['ACC'])['PROJ'].transform(lambda x: ', '.join(x))
temp.drop_duplicates(subset='PROJS',inplace=True)
temp.reset_index(inplace=True) # avoid NA columns

acctsummary['PROJS'] = temp['PROJS']
acctsummary

,ACC,PROJS
0,ANGELA,"3H aircon, Active Links Airconditioning Engrg,..."
1,CAIYEAN,"3H aircon, Additions Lifestyle, Aim Aircon Eng..."
2,CHWEELIAN,"Changi Sailing Club, VT Travel"
3,DINA,"American Career, Ariva Hospitality, Basketball..."
4,GERALDINE,"CGI Ind, Linear Tooling, Lingfine Jewellery, P..."
5,HOAYBEE,"Aidha Ltd, Alan Telecom, American Career, Bach..."
6,JASON,"Aidha Ltd, Al- us Enterprises, Cicada Privat..."
7,JOSEPH,"ADCT Technologies, Al- us Enterprises, Aldon..."
8,RAYMOND,"ADCT Technologies, Additions Lifestyle, Aim Ai..."
9,SEETLEE,"3H aircon, ADCT Technologies, Active Links Air..."


## 3.6 Final Productivity Indexes for each accountant

note: correlation between PI & seniority

In [16]:
acctt.sort_values(by=['RANK','PRODUCTIVITY']).drop('Accountant',axis=1)

,ACC,RANK,RATE,PRODUCTIVITY
0,ANGELA,Junior,15,31.60
8,RAYMOND,Junior,15,39.98
7,JOSEPH,Junior,15,40.91
6,JASON,Junior,15,42.85
5,HOAYBEE,Middle,25,38.62
11,SIEWJIN,Middle,25,47.18
2,CHWEELIAN,Middle,25,50.00
10,SERENE,Senior,35,48.00
4,GERALDINE,Senior,35,66.18
3,DINA,Senior,35,67.79


# ~ The End ~ (that's all for now!)